In [39]:
%reload_ext autoreload
%autoreload 2
import tie.dbutils as db
import tie.utils as qa
import pandas as pd
from scipy.io import savemat, loadmat
import json
from tqdm import tqdm
import os
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rc
import numpy as np
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import DivergingNorm, TwoSlopeNorm
from statsmodels.stats.multitest import fdrcorrection
import matplotlib.patches as patches

In [40]:
rc('font', **{'family':'sans-serif',
              'sans-serif':['Helvetica']})
FONT_SIZE = 25
params = {'axes.labelsize': FONT_SIZE,
          'axes.titlesize': FONT_SIZE, 
          'legend.fontsize': FONT_SIZE, 
          'xtick.labelsize': FONT_SIZE, 
          'ytick.labelsize': FONT_SIZE}
matplotlib.rcParams.update(params)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20, 10))
# df["days"].hist(bins=30, ax=ax[0])
full_info[full_info["days"] < 90]["days"].hist(bins=np.linspace(0, 90, 5), ax=ax[0][0])
full_info[(full_info["days"] >= 90) & (full_info["days"] < 180)]["days"].hist(bins=np.linspace(90, 180, 5), ax=ax[0][0])
full_info[(full_info["days"] >= 180) & (full_info["days"] < 365)]["days"].hist(bins=np.linspace(180, 365, 10), ax=ax[0][0])
full_info[full_info["days"] >= 365]["days"].hist(bins=np.linspace(365, 545, 10), ax=ax[0][0])

ax[0][0].set_xlabel("Days")
ax[0][0].set_ylabel("# Subjects")
ax[0][0].set_yscale('log')
ax[0][0].grid(False)
count_days = Counter()
count_days["n < 3 months"] = 0
count_days["3 months < n < 6 months"] = 0
count_days["6 months < n < 1 year"] = 0
count_days["n > 1 year"] = 0

for i in full_info['days'].values:
    if i > 365:
        count_days["n > 1 year"] += 1
        continue
    if i > 180:
        count_days["6 months < n < 1 year"] += 1
        continue
    if i > 90:
        count_days["3 months < n < 6 months"] += 1
        continue
    if i > 30:
        count_days["n < 3 months"] += 1
        continue
labels = list(count_days.keys())
sizes = list(count_days.values())
explode = (0.1, 0.1, 0.1, 0.1)  

ax[0][1].pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax[0][1].axis('equal');

## AGE

full_info[full_info["age"] < 20]["age"].hist(bins=np.linspace(0, 20, 10), ax=ax[1][0])
full_info[(full_info["age"] >= 20) & (full_info["age"] < 40)]["age"].hist(bins=np.linspace(20, 40, 10), ax=ax[1][0])
full_info[(full_info["age"] >= 40) & (full_info["age"] < 60)]["age"].hist(bins=np.linspace(40, 60, 10), ax=ax[1][0])
full_info[full_info["age"] >= 60]["age"].hist(bins=np.linspace(60, 100, 20), ax=ax[1][0])

ax[1][0].set_xlabel("Age")
ax[1][0].set_ylabel("# Subjects")
# ax[1][0].set_yscale('log')
ax[1][0].grid(False)
count_days = Counter()
count_days["< 20"] = 0
count_days["20 - 39"] = 0
count_days["40 - 59"] = 0
count_days["60+"] = 0

for i in full_info['age'].values:
    if i >= 60:
        count_days["60+"] += 1
        continue
    if i >= 40:
        count_days["40 - 59"] += 1
        continue
    if i >= 20:
        count_days["20 - 39"] += 1
        continue
    if i > 0:
        count_days["< 20"] += 1
        continue
labels = list(count_days.keys())
sizes = list(count_days.values())
explode = (0.1, 0.1, 0.1, 0.1)  

ax[1][1].pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax[1][1].axis('equal');

plt.tight_layout()
plt.savefig('figure_1_panel_a.pdf', transparent=True, bbox_to_inches=True)